In [39]:
!pip install vecstack
import pandas as pd
import numpy as np
from vecstack import stacking
import matplotlib.pyplot as plt
import seaborn as sns
color=sns.color_palette()
sns.set_style('darkgrid')
from scipy import stats
from scipy.stats import norm,skew
from vecstack import stacking
from sklearn.metrics import accuracy_score
from scipy.special import boxcox1p
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [24]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
#Read the train and test file
trainfile =r'/gdrive/My Drive/CIS_508/Datasets/Homeprice_train_processed.csv'
train=pd.read_csv(trainfile)
train.head()

testfile=r'/gdrive/My Drive/CIS_508/Datasets/Homeprice_test_processed.csv'
test=pd.read_csv(testfile)

wp_test=r'/gdrive/My Drive/CIS_508/Datasets/test_homeprice.csv'
wptest=pd.read_csv(wp_test)


In [26]:
y_train=train.iloc[:,-1]
x_train=train.iloc[:,:-1]

x_test=test

y_train.shape

(1460,)

In [0]:
#Validation function
n_folds = 5

def RMSE_CV(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x_train)
    rmse= np.sqrt(-cross_val_score(model, x_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)


In [38]:
params_knn={'n_neighbors':np.arange(5,20),'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree'],'leaf_size':np.arange(3,30)
           ,'p':np.arange(1,2)}

knn=KNeighborsRegressor()

knn_hp=RandomizedSearchCV(knn,params_knn,n_iter=3,cv=5,verbose=1)

%time knn_hp.fit(x_train,y_train)

print("best parameters are:",knn_hp.best_params_)

result=RMSE_CV(knn_hp)
print(result.mean())
print(result.std())

knn_hp.score(x_train,y_train)



Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    4.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 3.99 s, sys: 3.3 ms, total: 3.99 s
Wall time: 4 s
best parameters are: {'weights': 'uniform', 'p': 1, 'n_neighbors': 11, 'leaf_size': 6, 'algorithm': 'ball_tree'}
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.8s finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.3s finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.7s finished


0.1828101814416036
0.008627105239861339


0.8294457761463757

In [0]:
prediction=pd.DataFrame(knn_hp.predict(x_test),columns=["SalePrice"])

pd.concat([wptest['Id'],prediction['SalePrice']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_knn.csv', index = None)

In [40]:
#SVM

params_svm={'kernel':['rbf','poly','sigmoid'],'degree':np.arange(3,4,5)}

svm=SVR()

svm_hp=RandomizedSearchCV(svm,params_svm,n_iter=3,cv=5,verbose=2)

%time svm_hp.fit(x_train,y_train)

print("best parameters are:",svm_hp.best_params_)

result=RMSE_CV(svm_hp)
print(result.mean())
print(result.std())

svm_hp.score(x_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] kernel=rbf, degree=3 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................. kernel=rbf, degree=3, total=   0.4s
[CV] kernel=rbf, degree=3 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ............................. kernel=rbf, degree=3, total=   0.4s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.4s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.4s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.4s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.8s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.8s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.7s
[CV] kernel=poly, degree=3 ...........................................
[CV] .

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ............................. kernel=rbf, degree=3, total=   0.3s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.3s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.3s
[CV] kernel=rbf, degree=3 ............................................
[CV] ............................. kernel=rbf, degree=3, total=   0.3s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.5s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.5s
[CV] kernel=poly, degree=3 ...........................................
[CV] ............................ kernel=poly, degree=3, total=   0.4s
[CV] kernel=poly, degree=3 ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    6.4s finished


0.15067716307499082
0.010914859702782175


0.9642624459007724

In [11]:
#Lets perform STACKING- MODEL- 2

print('Ensembling gives the below results for MODEL-2:')


models_used = [GradientBoostingRegressor(), RandomForestRegressor(), MLPRegressor() ]
      
s_train, s_test = stacking(models_used, x_train, y_train, x_test, regression=True,mode='oof_pred_bag', needs_proba=False,save_dir=None, n_folds=3,  stratified=True, shuffle=True,random_state=0, verbose=2)
                                


Ensembling gives the below results for MODEL-2:
task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [0.08631751]
    fold  1:  [0.09613503]
    fold  2:  [0.08626074]
    ----
    MEAN:     [0.08957109] + [0.00464146]
    FULL:     [0.08957336]

model  1:     [RandomForestRegressor]
    fold  0:  [0.10255211]
    fold  1:  [0.11694480]
    fold  2:  [0.10364643]
    ----
    MEAN:     [0.10771444] + [0.00654212]
    FULL:     [0.10771723]

model  2:     [MLPRegressor]
    fold  0:  [0.13419726]
    fold  1:  [0.14603836]
    fold  2:  [0.13976548]
    ----
    MEAN:     [0.14000037] + [0.00483696]
    FULL:     [0.14000053]



In [21]:
#predictions after stacking

rfm=RandomForestRegressor(n_estimators=40,max_depth=15,
                          bootstrap=False, n_jobs=-1,max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=11, random_state =42)

%time rfm.fit(s_train,y_train)



result=RMSE_CV(rfm)
print(result.mean())
print(result.std())

rfm.score(s_train,y_train)

prediction=pd.DataFrame(rfm.predict(s_test),columns=["SalePrice"])

pd.concat([wptest['Id'],prediction['SalePrice']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_homeprice_model2.csv', index = None)

CPU times: user 105 ms, sys: 1.76 ms, total: 107 ms
Wall time: 125 ms
0.1680013150133449
0.012590523979151435


In [34]:
prediction=pd.DataFrame(rfm.predict(s_test),columns=["SalePrice"])

pd.concat([wptest['Id'],prediction['SalePrice']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_homeprice_model2.csv', index = None)

ValueError: ignored